# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

image_shape = [32, 32]
sampling_size = 550
total_data_size = 4
batch_size = 1

def load():
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    return image

def generator():
    for _ in range(total_data_size):
        yield load()
        
feature_pipe = tf.data.Dataset.from_generator(generator, tf.float32).batch(batch_size)

# 2. Build Anomaly Detector

2-1. Feature-Based

In [4]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
out = tfdet.model.backbone.wide_resnet50_2_torch(x, weights = "imagenet", indices = [0, 1, 2])
model = tf.keras.Model(x, out)
feature = model.predict(feature_pipe)

n_feature = np.sum([np.shape(f)[-1] for f in feature])
sampling_index = np.random.choice(np.arange(n_feature), sampling_size, replace = False)
feature_vector = tfdet.model.train.padim.train(feature, sampling_index = sampling_index, memory_reduce = False) #memory_reduce is a tradeoff between accuracy and memory

score, mask = tfdet.model.detector.padim(out, feature_vector, image_shape = image_shape, sampling_index = sampling_index, memory_reduce = False) #align memory_reduce with train in test
filtered_out = tfdet.model.postprocess.padim.FilterDetection(threshold = 0.5)([score, mask]) #threshold > tfdet.util.get_threshold(label, pred)

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(feature_pipe, verbose = 0)]

4/4 [==============================] - 1s 174ms/step


[(4, 1), (4, 32, 32, 1)]

In [5]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
out = tfdet.model.backbone.wide_resnet50_2_torch(x, weights = "imagenet", indices = [0, 1, 2])
model = tf.keras.Model(x, out)
feature = model.predict(feature_pipe)

n_feature = np.sum([np.shape(f)[-1] for f in feature])
sampling_index = np.random.choice(np.arange(n_feature), sampling_size, replace = False)
feature_vector = tfdet.model.train.patch_core.train(feature, sampling_index = sampling_index, memory_reduce = False) #memory_reduce is a tradeoff between accuracy and memory

score, mask = tfdet.model.detector.patch_core(out, feature_vector, image_shape = image_shape, k = 9, sampling_index = sampling_index, memory_reduce = False) #align memory_reduce with train in test
filtered_out = tfdet.model.postprocess.patch_core.FilterDetection(threshold = 0.5)([score, mask]) #threshold > tfdet.util.get_threshold(label, pred)

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(feature_pipe, verbose = 0)]

4/4 [==============================] - 0s 14ms/step


[(4, 1), (4, 32, 32, 1)]

In [6]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
out = tfdet.model.backbone.wide_resnet50_2_torch(x, weights = "imagenet", indices = [0, 1, 2])
model = tf.keras.Model(x, out)
feature = model.predict(feature_pipe)

n_feature = np.sum([np.shape(f)[-1] for f in feature])
sampling_index = np.random.choice(np.arange(n_feature), sampling_size, replace = False)
feature_vector = tfdet.model.train.spade.train(feature, sampling_index = sampling_index) #memory_reduce is a tradeoff between accuracy and memory

score, mask = tfdet.model.detector.spade(out, feature_vector, image_shape = image_shape, k = 50, sampling_index = sampling_index) #align memory_reduce with train in test
filtered_out = tfdet.model.postprocess.spade.FilterDetection(threshold = 0.5)([score, mask]) #threshold > tfdet.util.get_threshold(label, pred)

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(feature_pipe, verbose = 0)]

4/4 [==============================] - 0s 14ms/step


[(4, 1), (4, 32, 32, 1)]